### Tutorial for tokenizer

In [1]:
import os,sys
sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..')))
# import warnings
# warnings.filterwarnings("ignore")
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from SpatialGPT.config import config
opt = config()
args = opt.parse_args(['--gpu', '0'])
args.dataset_path = '../Data/DLPFC/'
args.max_epoch = 30
args

Namespace(amp=True, att_mask=True, batch_size=32, clusters=0, d_model=100, dataset_path='../Data/DLPFC/', decoder='NB', dropout=0.2, early_stop=True, eval=False, fast=True, flow='source_to_target', gpu=0, hvgs=1200, latent_dim=64, log_file='../Log', log_steps=10, lr=0.0001, max_epoch=30, max_neighbors=6, n_gcn=3, n_head=8, n_hops=3, n_layer=4, n_randomwalk=[5, 10, 15], n_randwalk=60, out_file='../Output', project_name='SpatialGRN', q_randomwalk=[1, 1.5], schedule_ratio=0.9, seed=2024, srt_resolution=2, tau=1, version='1.0', visual=True, wegiht_decay=1e-06)

In [3]:
from SpatialGPT.utils import set_random_seed
set_random_seed(args.seed)

**Read data**

In [4]:
sector = '151507'
adata = sc.read_visium(os.path.join(args.dataset_path+sector))
adata

/home/zl/anaconda3/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zl/anaconda3/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

**Token generation**

In [5]:
from SpatialGPT.tokenizer import SpotTokenizer


In [6]:
from SpatialGPT.tokenizer import GeneID


In [13]:
from pathlib import Path
path = Path('/home/zl/SpatialGRN/Data/default_census_vocab.json')
import json
with path.open('r') as f:
    tokens = json.load(f)
vocab = GeneID.from_file(path)
'<pad>' in vocab, len(vocab)

(True, 60697)

In [46]:
adata.var["gene_name"] = adata.var.index.tolist()
adata.var["id_in_vocab"] = [
        1 if gene in vocab else -1 for gene in adata.var["gene_name"]
    ]
gene_ids_in_vocab = np.array(adata.var["id_in_vocab"])
np.sum(gene_ids_in_vocab >= 0)/len(gene_ids_in_vocab)

0.6814359830639871

In [52]:
adata

AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome', 'gene_name', 'id_in_vocab'
    uns: 'spatial'
    obsm: 'spatial'

In [9]:
import numpy as np
a=[5, 15]
for i in range(a[0], a[1], 2):
    print(i)

5
7
9
11
13
